## Read files

In [5]:
import re
from tqdm import tqdm
import pandas as pd
import glob
import pickle
import gc
import os
import re
import pickle
import pandas as pd
import numpy as np
import dask.dataframe as dd
from tqdm import tqdm

# Read the recommendation_id.pickle file
with open(os.path.join('recommendation_id.pkl'), 'rb') as file:
    recommendation_id = pickle.load(file)
print(len(recommendation_id))



def sort_key(s):
    numbers = re.findall(r'\d+', s)
    return int(numbers[0]) if numbers else 0

# Read the recommendation_id.pickle file
with open(os.path.join('recommendation_id.pkl'), 'rb') as file:
    recommendation_id = pickle.load(file)

directory = '.'  # Current directory
pickle_files = [file for file in os.listdir(directory) if file.endswith('.pickle') and file != 'recommendation_id.pkl']
pickle_files.sort(key=sort_key)


920536


## Convert them into csv files (in parts due to their large size)

In [58]:

def convert_and_save_df(similarities, recommendation_ids_filtered, accounted_for):
    # Filter similarities and recommendation_ids_filtered based on accounted_for
    similarities_filtered = []
    recommendation_ids_filtered_new = []
    for sub_lst, val in zip(similarities, recommendation_ids_filtered):
        if val not in accounted_for:
            similarities_filtered.append(sub_lst)
            recommendation_ids_filtered_new.append(val)

    # Return an empty dataframe if no suitable values found
    if len(recommendation_ids_filtered_new)==0:
        return (pd.DataFrame(dtype='float64'))

    # Cache the lengths of the sublists in similarities_filtered
    lengths = [len(sub_lst) for sub_lst in similarities_filtered]

    # Flatten similarities_filtered using the cached lengths
    similarities = np.concatenate(similarities_filtered)

    # Repeat elements in recommendation_ids_filtered according to the cached lengths
    recommendation_id = np.repeat(recommendation_ids_filtered_new, lengths)

    # Create a DataFrame from the columns
    df = pd.DataFrame({'similarities': similarities, 'recommendation_id': recommendation_id})
    return df


length, accounted_for, i = 0, [], 1
for filename in tqdm(pickle_files):
    # Load the current file
    with open(filename, 'rb') as file:
        current = pickle.load(file)
    new_length = len(current)+length
    recommendation_ids_filtered = (recommendation_id[length:new_length])
    df = convert_and_save_df(current, recommendation_ids_filtered, accounted_for)
    accounted_for += recommendation_ids_filtered
    length = new_length
    if df.empty:
        continue
        #print(filename)
    else:
        df.to_csv(f'similarities_control_part_{i}.csv', index=False)
        
    i = i + 1



 41%|███████████████▊                       | 356/876 [1:18:14<38:16,  4.42s/it]

control_matrix_400000_to_401000.pickle


 41%|███████████████▉                       | 357/876 [1:18:18<37:02,  4.28s/it]

control_matrix_401000_to_402000.pickle


 41%|███████████████▉                       | 358/876 [1:18:22<35:38,  4.13s/it]

...

 79%|██████████████████████████████▋        | 689/876 [2:03:12<23:24,  7.51s/it]

control_matrix_733000_to_734000.pickle


 79%|██████████████████████████████▋        | 690/876 [2:03:19<23:12,  7.49s/it]

control_matrix_734000_to_735000.pickle


100%|███████████████████████████████████████| 876/876 [2:33:05<00:00, 10.49s/it]


## Save them in sizeable csv files

In [3]:


# Get a list of all CSV files in the current directory
csv_files = glob.glob('*.csv')

# Create a generator that will load and yield dataframes
def load_csvs(file_list):
    for file in file_list:
        yield pd.read_csv(file)

# Use the generator to load and concatenate CSV files in chunks
num_files = len(csv_files)
chunk_size = num_files // 10  # Number of files per output CSV
remainder = num_files % 10  # Remaining files for the last CSV
chunks = [csv_files[i * chunk_size:(i + 1) * chunk_size] for i in tqdm(range(10))]
if remainder > 0:
    chunks[-1] += csv_files[-remainder:]  # Add remaining files to the last chunk

# Concatenate and save each chunk
for i, chunk in tqdm(enumerate(chunks, 1)):
    df = pd.concat(load_csvs(chunk))
    df.to_csv(f'combined_control_ids_part_{i}.csv', index=False)